In [3]:
import numpy as np
import mindspore.nn as nn
import mindspore as ms

# 构建多标签数据集

In [4]:
import numpy as np
from mindspore import dataset as ds

def get_multilabel_data(num, w=2.0, b=3.0):
    for _ in range(num):
        x = np.random.uniform(-10.0, 10.0)
        noise1 = np.random.normal(0, 1)
        noise2 = np.random.normal(-1, 1)
        y1 = x * w + b + noise1
        y2 = x * w + b + noise2
        yield np.array([x]).astype(np.float32), np.array([y1]).astype(np.float32), np.array([y2]).astype(np.float32)

def create_multilabel_dataset(num_data, batch_size=16):
    dataset = ds.GeneratorDataset(list(get_multilabel_data(num_data)), column_names=['data', 'label1', 'label2'])
    dataset = dataset.batch(batch_size)  # 每个batch有16个数据
    return dataset


# 多标签损失函数

In [5]:
class MAELossForMultiLabel(nn.LossBase):
    def __init__(self, reduction="mean"):
        super(MAELossForMultiLabel, self).__init__(reduction)
        self.abs = ops.Abs()

    def construct(self, base, target1, target2):
        x1 = self.abs(base - target1)
        x2 = self.abs(base - target2)
        return (self.get_loss(x1) + self.get_loss(x2))/2


# 定义损失函数

In [6]:
class CustomWithLossCell(nn.Cell):
    def __init__(self, backbone, loss_fn):
        super(CustomWithLossCell, self).__init__(auto_prefix=False)
        self._backbone = backbone
        self._loss_fn = loss_fn

    def construct(self, data, label1, label2):
        output = self._backbone(data)
        return self._loss_fn(output, label1, label2)


# 定义网络模型并训练

In [11]:
from mindspore.common.initializer import Normal
import mindspore.ops as ops
from mindvision.engine.callback import LossMonitor
class LinearNet(nn.Cell):
    """定义线性回归网络"""
    def __init__(self):
        super(LinearNet, self).__init__()
        self.fc = nn.Dense(1, 1, Normal(0.02), Normal(0.02))

    def construct(self, x):
        return self.fc(x)
    
ds_train = create_multilabel_dataset(num_data=160)
net = LinearNet()

# 定义多标签损失函数
loss = MAELossForMultiLabel()

# 定义损失网络，连接前向网络和多标签损失函数
loss_net = CustomWithLossCell(net, loss)

# 定义优化器
opt = nn.Momentum(net.trainable_params(), learning_rate=0.005, momentum=0.9)

# 定义Model，多标签场景下Model无需指定损失函数
model = ms.Model(network=loss_net, optimizer=opt)

model.train(epoch=1, train_dataset=ds_train, callbacks=[LossMonitor(0.005)])


[WARNING] ME(5924:12492,MainProcess):2022-12-23-16:01:10.939.243 [mindspore\dataset\engine\datasets_user_defined.py:656] Python multiprocessing is not supported on Windows platform.


Epoch:[  0/  1], step:[    1/   10], loss:[11.166/11.166], time:882.640 ms, lr:0.00500
Epoch:[  0/  1], step:[    2/   10], loss:[11.010/11.088], time:17.951 ms, lr:0.00500
Epoch:[  0/  1], step:[    3/   10], loss:[10.315/10.830], time:0.999 ms, lr:0.00500
Epoch:[  0/  1], step:[    4/   10], loss:[11.732/11.056], time:3.002 ms, lr:0.00500
Epoch:[  0/  1], step:[    5/   10], loss:[9.359/10.717], time:4.988 ms, lr:0.00500
Epoch:[  0/  1], step:[    6/   10], loss:[6.328/9.985], time:0.998 ms, lr:0.00500
Epoch:[  0/  1], step:[    7/   10], loss:[7.772/9.669], time:1.994 ms, lr:0.00500
Epoch:[  0/  1], step:[    8/   10], loss:[6.167/9.231], time:5.984 ms, lr:0.00500
Epoch:[  0/  1], step:[    9/   10], loss:[9.109/9.218], time:2.992 ms, lr:0.00500
Epoch:[  0/  1], step:[   10/   10], loss:[5.866/8.883], time:0.996 ms, lr:0.00500
Epoch time: 955.423 ms, per step time: 95.542 ms, avg loss: 8.883
